## Stabilization, P = 1.4

In [1]:
#-----Import some essential packages

from __future__ import division
import hoomd
import hoomd.md

#-----Some important variables

p_max = 1.4;
t_max = 5.2;
steps = 1e5;

#-----Crea un contexto de simulación

hoomd.context.initialize("--mode=cpu", memory_traceback = True);

#-----Extrae la configuración del centro de masa de las partículas

system = hoomd.init.read_gsd("T_CM_5.2_P_1.4_ramp.gsd",frame = -1)

#-----Define cada mesógeno en el marco de referencia local de cada centro de masa.

rigid = hoomd.md.constrain.rigid();
rigid.set_param('M', 
                types=['A']*8,
                positions=[(-4,0,0),(-3,0,0),(-2,0,0),(-1,0,0),
                           (1,0,0),(2,0,0),(3,0,0),(4,0,0)]);

#-----Declara que las moléculas creadas son cuerpos rígidos

rigid.create_bodies();

#-----Define la energía potencial

nl = hoomd.md.nlist.tree();

lj = hoomd.md.pair.lj(r_cut = 3.5, nlist = nl)
lj.set_params(mode = 'shift')

#-----Define la interacción entre las especies de la simulación

lj.pair_coeff.set('M','M', epsilon = 1.0, sigma = 1.0)
lj.pair_coeff.set('A','A', epsilon = 1.0, sigma = 1.0)
lj.pair_coeff.set('M','A', epsilon = 1.0, sigma = 1.0)

#-----Selecciona un integrador standard

hoomd.md.integrate.mode_standard(dt = 0.005);

#------Define el grupo sobre el cual se integraran las ecuaciones de movimiento

mesogens = hoomd.group.rigid_center();

HOOMD-blue 2.6.0 DOUBLE HPMC_MIXED TBB SSE SSE2 SSE3 
Compiled: 05/29/2019
Copyright (c) 2009-2019 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, C D Lorenz, and A Travesset. "General purpose molecular dynamics
  simulations fully implemented on graphics processing units", Journal of
  Computational Physics 227 (2008) 5342--5359
* J Glaser, T D Nguyen, J A Anderson, P Liu, F Spiga, J A Millan, D C Morse, and
  S C Glotzer. "Strong scaling of general-purpose molecular dynamics simulations
  on GPUs", Computer Physics Communications 192 (2015) 97--107
-----
HOOMD-blue is running on the CPU
notice(2): Group "all" created containing 1000 particles
notice(2): constrain.rigid(): Creating 1000 rigid bodies (adding 8000 particles)
-----
You are using tree neighbor lists. Please cite the following:
* M P Howard, J A Anderson, A Nikoubashman, S C Glotzer, and A Z
  Panagiotopoulos. "Efficient neighbor list calculation for mol

In [2]:
#----Integrate using NPT

npt = hoomd.md.integrate.npt(group = mesogens, kT = t_max, tau = 10.0, tauP = 10.0, P = p_max);
npt.randomize_velocities(seed = 42)

#-----Write output and Run the Simulation

log_file = "T_" + str(t_max) + "_P_" + str(p_max) + "_equilibrium.log"
gsd_file = "T_" + str(t_max) + "_P_" + str(p_max) + "_equilibrium.gsd" 
meso_gsd_file = "T_CM_" + str(t_max) + "_P_" + str(p_max) + "_equilibrium.gsd"

log = hoomd.analyze.log(filename = log_file,
                         quantities = ['num_particles',
                                     'ndof',
                                     'translational_ndof',
                                     'rotational_ndof',
                                     'potential_energy',
                                     'kinetic_energy',
                                     'translational_kinetic_energy',
                                     'rotational_kinetic_energy',
                                     'temperature',
                                     'pressure',
                                      'volume'],
                         period = 1e3,
                         overwrite = True);
gsd = hoomd.dump.gsd(gsd_file,
               period = 1e3,
               group = hoomd.group.all(),
               overwrite = True); 
meso_gsd = hoomd.dump.gsd(meso_gsd_file,
               period = 1e3,
               group = mesogens,
               overwrite = True); 

In [3]:
hoomd.run(steps)

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 9000
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: yes
** starting run **
Time 00:00:10 | Step 1054149 / 1154000 | TPS 14.8877 | ETA 01:51:46
Time 00:00:20 | Step 1054304 / 1154000 | TPS 15.4352 | ETA 01:47:38
Time 00:00:30 | Step 1054463 / 1154000 | TPS 15.8591 | ETA 01:44:36
Time 00:00:40 | Step 1054638 / 1154000 | TPS 17.414 | ETA 01:35:05
Time 00:00:50 | Step 1054820 / 1154000 | TPS 18.1396 | ETA 01:31:07
Time 00:01:00 | Step 1055004 / 1154000 | TPS 18.2499 | ETA 01:30:24
Time 00:01:10 | Step 1055181 / 1154000 | TPS 17.6539 | ETA 01:33:17
Time 00:01:20 | Step 1055344 / 1154000 | TPS 16.2266 | ETA 01:41:19
Time 00:01:30 | Step 1055514 / 1154000 | TPS 16.9585 | ETA 01:36:47
Time 00:01:40 | Step 1055690 / 1154000 | TPS 17.4301 | ETA 01:34:00
Time 00:01:50 | Step 1055876 / 1154000 | TPS 18.5172 | ETA 01:28:19
Time 00:02

In [4]:
system.get_metadata()

OrderedDict([('box', <hoomd.data.boxdim at 0x105d6d7f0>),
             ('particles', <hoomd.data.particle_data at 0x1079e2b38>),
             ('number_density', 0.715725678095504),
             ('bonds', <hoomd.data.bond_data at 0x1079e2ba8>),
             ('angles', <hoomd.data.angle_data at 0x1079e2c18>),
             ('dihedrals', <hoomd.data.dihedral_data at 0x1079e2c88>),
             ('impropers', <hoomd.data.dihedral_data at 0x1079e2cf8>),
             ('constraints', <hoomd.data.constraint_data at 0x1079e2d68>),
             ('pairs', <hoomd.data.bond_data at 0x1079e2dd8>),
             ('timestep', 1154000)])

In [5]:
npt.set_params(tau = 10.5, tauP = 10.5);
hoomd.run(steps)

** starting run **
Time 01:33:06 | Step 1154179 / 1254000 | TPS 17.7834 | ETA 01:33:33
Time 01:33:16 | Step 1154361 / 1254000 | TPS 18.1345 | ETA 01:31:34
Time 01:33:26 | Step 1154547 / 1254000 | TPS 18.5627 | ETA 01:29:17
Time 01:33:36 | Step 1154726 / 1254000 | TPS 17.7714 | ETA 01:33:06
Time 01:33:46 | Step 1154908 / 1254000 | TPS 18.0312 | ETA 01:31:35
Time 01:33:56 | Step 1155087 / 1254000 | TPS 17.8799 | ETA 01:32:12
Time 01:34:06 | Step 1155267 / 1254000 | TPS 17.9474 | ETA 01:31:41
Time 01:34:16 | Step 1155455 / 1254000 | TPS 18.7525 | ETA 01:27:35
Time 01:34:26 | Step 1155639 / 1254000 | TPS 18.3891 | ETA 01:29:08
Time 01:34:36 | Step 1155822 / 1254000 | TPS 18.2874 | ETA 01:29:28
Time 01:34:46 | Step 1156004 / 1254000 | TPS 18.0253 | ETA 01:30:36
Time 01:34:56 | Step 1156187 / 1254000 | TPS 18.1322 | ETA 01:29:54
Time 01:35:06 | Step 1156370 / 1254000 | TPS 18.1994 | ETA 01:29:24
Time 01:35:16 | Step 1156555 / 1254000 | TPS 18.4639 | ETA 01:27:57
Time 01:35:26 | Step 1156741 

In [6]:
system.get_metadata()

OrderedDict([('box', <hoomd.data.boxdim at 0x105d5b128>),
             ('particles', <hoomd.data.particle_data at 0x1079e2b38>),
             ('number_density', 0.7143476860189519),
             ('bonds', <hoomd.data.bond_data at 0x1079e2ba8>),
             ('angles', <hoomd.data.angle_data at 0x1079e2c18>),
             ('dihedrals', <hoomd.data.dihedral_data at 0x1079e2c88>),
             ('impropers', <hoomd.data.dihedral_data at 0x1079e2cf8>),
             ('constraints', <hoomd.data.constraint_data at 0x1079e2d68>),
             ('pairs', <hoomd.data.bond_data at 0x1079e2dd8>),
             ('timestep', 1254000)])